In [1]:
import os
import pandas as pd
import numpy as np
import statistics as stat
import seaborn as sns
import matplotlib.pyplot as plt
from statistics import mode as py_mode

In [2]:
pwd

'C:\\Users\\Yudith\\Desktop\\Análisis de Datos\\DATA ANALYTICS'

In [3]:
df = pd.read_csv(
    r"C:\Users\Yudith\Desktop\Análisis de Datos\DATA ANALYTICS\ventas_supermercado_argentina_2024_2025.csv",
    encoding="utf-8-sig"
)


In [4]:
df.head()

,id_pedido,id_cliente,fecha,hora,id_departamento,id_seccion,id_producto,nombre_producto,precio_unitario,cantidad,precio_total
0,347621,5824,2024-05-20,11,1,2,1002,Manzana Roja,2309.03,5,11545.15
1,202790,13674,2025-01-17,8,1,2,1004,Leche Entera,1831.60,1,1831.60
2,327127,7257,2025-04-23,15,3,3,1001,Banana,1906.34,2,3812.68
3,290785,9574,2025-03-21,11,2,1,1002,Manzana Roja,2041.93,3,6125.79
4,270165,13891,2025-01-23,19,2,5,1002,Manzana Roja,2575.80,4,10303.20


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_pedido        6000 non-null   int64  
 1   id_cliente       6000 non-null   int64  
 2   fecha            6000 non-null   object 
 3   hora             6000 non-null   int64  
 4   id_departamento  6000 non-null   int64  
 5   id_seccion       6000 non-null   int64  
 6   id_producto      6000 non-null   int64  
 7   nombre_producto  6000 non-null   object 
 8   precio_unitario  6000 non-null   float64
 9   cantidad         6000 non-null   int64  
 10  precio_total     6000 non-null   float64
dtypes: float64(2), int64(7), object(2)
memory usage: 515.8+ KB


LA FECHA SALE COMO OBJECT (TEXTO) HAY QUE PASARLO A NÚMERICA. EN ESTE ESTUDIO NO ES NECESARIO PORQUE NO LA VAMOS A USAR, PERO EN ANÁLISIS DE VENTAS SI DEBEMOS TENERLO EN CUENTA 

PARA CONVERTIR LA FECHA DE TEXTO A NÚMERO

In [7]:
df['fecha'] = pd.to_datetime(df['fecha'])


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_pedido        6000 non-null   int64         
 1   id_cliente       6000 non-null   int64         
 2   fecha            6000 non-null   datetime64[ns]
 3   hora             6000 non-null   int64         
 4   id_departamento  6000 non-null   int64         
 5   id_seccion       6000 non-null   int64         
 6   id_producto      6000 non-null   int64         
 7   nombre_producto  6000 non-null   object        
 8   precio_unitario  6000 non-null   float64       
 9   cantidad         6000 non-null   int64         
 10  precio_total     6000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(7), object(1)
memory usage: 515.8+ KB


LA  FECHA SE CONVIRTIÓ A VARIABLE NUMÉRICA

In [ ]:
df.isnull().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
def test_normalidad_economica(df, columnas, sample_size=5000):
    """
    Prueba de normalidad para variables económicas.
    Usa Shapiro-Wilk si la muestra es pequeña,
    y D'Agostino-Pearson si es grande.
    """

    import pandas as pd
    import numpy as np
    from scipy import stats

    resultados = []

    for col in columnas:
        # Limpiar datos
        data = df[col].dropna()
        data = data[np.isfinite(data)]

        # Validación mínima
        if len(data) < 8:
            resultados.append({
                'Variable': col,
                'Test': None,
                'Estadístico': None,
                'p-valor': None,
                'Conclusión': '⚠️ Datos insuficientes'
            })
            continue

        # Submuestreo para evitar sesgos con n grandes
        if len(data) > sample_size:
            data_test = data.sample(sample_size, random_state=42)
        else:
            data_test = data

        # Elegir test según tamaño muestral
        if len(data_test) <= 5000:
            stat, p = stats.shapiro(data_test)
            test_usado = "Shapiro-Wilk"
        else:
            stat, p = stats.normaltest(data_test)
            test_usado = "D'Agostino-Pearson"

        # Interpretación estadística
        if p < 0.05:
            conclusion = "❌ No sigue una distribución normal"
        else:
            conclusion = "✅ Sigue una distribución normal"

        resultados.append({
            'Variable': col,
            'Test': test_usado,
            'Estadístico': round(stat, 4),
            'p-valor': round(p, 6),
            'Conclusión': conclusion
        })

    return pd.DataFrame(resultados)


In [ ]:
columnas_analisis = [
    "precio_unitario",
    "cantidad",
    "precio_total"
]

resultados_normalidad = test_normalidad_economica(df, columnas_analisis)
resultados_normalidad


In [ ]:
def calcular_bins(df, variable):
    import numpy as np

    datos = df[variable].dropna()
    datos = datos[np.isfinite(datos)]
    n = len(datos)

    if n < 2:
        return {
            "Variable": variable,
            "n": n,
            "Rango": None,
            "IQR": None,
            "Sturges": None,
            "Raíz_cuadrada": None,
            "Freedman-Diaconis": None,
            "Nota": "⚠️ Datos insuficientes"
        }

    # Medidas básicas
    rango = datos.max() - datos.min()
    q75, q25 = np.percentile(datos, [75, 25])
    iqr = q75 - q25

    # Reglas para número de bins
    bins_sturges = int(np.ceil(1 + np.log2(n)))
    bins_sqrt = int(np.ceil(np.sqrt(n)))

    # Freedman–Diaconis (robusta a outliers)
    if iqr > 0:
        bin_width = 2 * iqr * n ** (-1/3)
        bins_fd = int(np.ceil(rango / bin_width))
    else:
        bins_fd = bins_sturges

    return {
        "Variable": variable,
        "n": n,
        "Rango": round(float(rango), 2),
        "IQR": round(float(iqr), 2),
        "Sturges": bins_sturges,
        "Raíz_cuadrada": bins_sqrt,
        "Freedman-Diaconis": bins_fd
    }


In [ ]:
import pandas as pd

variables_numericas = df.select_dtypes(include=["int64", "float64"]).columns

tabla_bins = pd.DataFrame(
    [calcular_bins(df, var) for var in variables_numericas]
)

tabla_bins


In [ ]:
variables_numericas = df.select_dtypes(include=['int64', 'float64']).columns

for variable in variables_numericas:

    # Histograma
    plt.figure(figsize=(8,5))
    sns.histplot(df[variable].dropna(), bins=15, kde=True)
    plt.title(f'Distribución de {variable}')
    plt.xlabel(variable)
    plt.ylabel('Frecuencia')
    plt.grid(axis='y', alpha=0.3)
    plt.show()

    # Boxplot
    plt.figure(figsize=(6,3))
    sns.boxplot(x=df[variable].dropna())
    plt.title(f'Boxplot de {variable}')
    plt.xlabel(variable)
    plt.grid(axis='x', alpha=0.3)
    plt.show()


In [ ]:
def detectar_outliers_precio_total(df):
    Q1 = df['precio_total'].quantile(0.25)
    Q3 = df['precio_total'].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    outliers = df[
        (df['precio_total'] < limite_inferior) |
        (df['precio_total'] > limite_superior)
    ]

    return outliers, limite_inferior, limite_superior


In [ ]:
outliers_precio, li, ls = detectar_outliers_precio_total(df)


In [ ]:
print("🔹 Variable: precio_total")
print(f"   Límite inferior: {round(li, 2)}")
print(f"   Límite superior: {round(ls, 2)}")
print(f"   Outliers detectados: {len(outliers_precio)}")


In [ ]:
if not outliers_precio.empty:
    print("\nValores más altos de precio_total:")
    display(
        outliers_precio[['precio_unitario', 'cantidad', 'precio_total']]
        .sort_values(by='precio_total', ascending=False)
        .head(10)
    )
else:
    print("No se detectaron outliers.")


In [ ]:
resumen_outliers_precio_total = pd.DataFrame([{
    'Variable': 'precio_total',
    'Outliers_detectados': len(outliers_precio),
    'Limite_inferior': round(li, 2),
    'Limite_superior': round(ls, 2)
}])

resumen_outliers_precio_total


La variable precio_total presenta una distribución asimétrica a la derecha, típica de variables económicas. Los valores atípicos detectados corresponden a transacciones de mayor volumen y no a errores de carga, por lo que se consideran observaciones válidas.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# =========================
# 1. CARGAR DATASET CSV
# =========================
ruta_csv = r"C:\Users\Yudith\Desktop\Análisis de Datos\DATA ANALYTICS\ventas_supermercado_argentina_2024_2025.csv"

df = pd.read_csv(ruta_csv)

# =========================
# 2. CONEXIÓN A MYSQL
# =========================
user = 'python_user'
password = 'MiPassword123!'
host = 'localhost'
database = 'worldanalysis'

connection_url = f"mysql+pymysql://{user}:{password}@{host}/{database}"
engine = create_engine(connection_url)

# =========================
# 3. EXPORTAR DATASET A SQL
# =========================
table_name = 'ventas_supermercado_argentina_2024_2025'

try:
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists='replace',  # reemplaza la tabla si existe
        index=False
    )
    print(f"✅ La tabla '{table_name}' fue creada o reemplazada correctamente.")
except Exception as e:
    print(f"❌ Error al crear la tabla '{table_name}':", e)
finally:
    engine.dispose()  # Cierra conexión
